In [24]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.autograd import Variable
import torchvision
import pathlib
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from IPython.display import clear_output
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [25]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [26]:
train_path = '/kaggle/input/oasis-brain/TRAIN/TRAIN/'
test_path = '/kaggle/input/oasis-brain/TEST/TEST/test/'

In [27]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [45]:
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transform),
    batch_size=32, shuffle=True,
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transform),
    batch_size=32, shuffle=True
)

In [29]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    model.train()
    correct = 0
    total = 0

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}, Train Accuracy: {accuracy:.2f}%")

In [30]:
def test(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

My model

In [40]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc = nn.Linear(32 * 26 * 26, num_classes)
        # self.lsm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        # out = self.lsm(out)
        return out

In [41]:
model = ConvNet(num_classes=4).to(device)
train(model, train_loader, epochs=1, learning_rate=0.001, device=device)
test_accuracy_light_ce = test(model, test_loader, device)

Epoch 1/1, Loss: 0.23554258649380178, Train Accuracy: 92.53%
Test Accuracy: 70.61%


In [12]:
torch.save(model.state_dict(), '/kaggle/working/convnet.pth')

Trial 2 finished with value: 63.4375 and parameters: {'n_layers': 5, '0 layer output channel': 16, '1 layer output channel': 29, '2 layer output channel': 36, '3 layer output channel': 45, '4 layer output channel': 64, 'lr': 5.973592490223275e-05}. Best is trial 2 with value: 63.4375.

In [43]:
class ConvNet_1(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet_1, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=36, kernel_size=3),
            nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=36, out_channels=45, kernel_size=3),
            nn.BatchNorm2d(45),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=45, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc = nn.Linear(64*12*12, num_classes)
        #self.lsm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        #out = self.lsm(out)
        return out

In [46]:
model1 = ConvNet_1(num_classes=4).to(device)
train(model1, train_loader, epochs=1, learning_rate=5.973592490223275e-05, device=device)
test_accuracy_light_ce = test(model1, test_loader, device)

Epoch 1/1, Loss: 0.27498661869873875, Train Accuracy: 90.31%
Test Accuracy: 69.66%


In [15]:
torch.save(model1.state_dict(), '/kaggle/working/convnet_1.pth')

Trial 7 finished with value: 62.1875 and parameters: {'n_layers': 3, '0 layer output channel': 22, '1 layer output channel': 34, '2 layer output channel': 48, 'lr': 0.003775760161262146}. Best is trial 7 with value: 62.1875.

In [20]:
class ConvNet_2(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet_2, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=22, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=34, kernel_size=3),
            nn.BatchNorm2d(34),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=34, out_channels=48, kernel_size=3),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.fc = nn.Linear(48*26*26, num_classes)
        #self.lsm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        #out = self.lsm(out)
        return out

In [21]:
model2 = ConvNet_2(num_classes=4).to(device)
train(model2, train_loader, epochs=1, learning_rate=0.003775760161262146, device=device)
test_accuracy_light_ce = test(model2, test_loader, device)

Epoch 1/1, Loss: 0.4767655500499866, Train Accuracy: 90.26%
Test Accuracy: 66.33%


In [22]:
torch.save(model2.state_dict(), '/kaggle/working/convnet_2.pth')

In [ ]:
Видим, что наилучший результат у model

In [35]:
class ConvNet_best(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet_best, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc = nn.Linear(32 * 26 * 26, num_classes)
        # self.lsm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        # out = self.lsm(out)
        return out

In [31]:
class ClassificationUnet(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationUnet, self).__init__()
        self.unet = smp.Unet(encoder_name="efficientnet-b0",
                             encoder_weights='imagenet',
                             in_channels=3,
                             classes=num_classes)

    def forward(self, x):
        out = self.unet(x)
        out = torch.mean(out, dim=(2, 3))
        return out

In [32]:
nn_deep = ClassificationUnet(num_classes=4).to(device)
#nn_deep.load_state_dict(torch.load('/kaggle/input/deep_my/pytorch/deep_my/1/nn_deep.pth', map_location=torch.device('cuda')))
train(nn_deep, train_loader, epochs=1, learning_rate=0.001, device=device)
test_accuracy_deep = test(nn_deep, test_loader, device)

Epoch 1/1, Loss: 0.2111385543391096, Train Accuracy: 92.04%
Test Accuracy: 71.88%


In [33]:
torch.save(nn_deep.state_dict(), '/kaggle/working/nn_deep_2.pth')

In [36]:
torch.manual_seed(42)
new_nn_light = ConvNet_best(num_classes=4).to(device)

In [37]:

total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in new_nn_light.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 6,251,904
LightNN parameters: 92,660


In [38]:
def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight, device):
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(student.parameters(), lr=learning_rate)

    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with the teacher model - do not save gradients here as we do not change the teacher's weights
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with the student model
            student_logits = student(inputs)

            #Soften the student logits by applying softmax first and log() second
            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft targets loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = -torch.sum(soft_targets * soft_prob) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [39]:
# Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
train_knowledge_distillation(teacher=nn_deep, student=new_nn_light, train_loader=train_loader, epochs=1, learning_rate=0.001, T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)
test_accuracy_light_ce_and_kd = test(new_nn_light, test_loader, device)

# Compare the student test accuracy with and without the teacher, after distillation
print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")
     

Epoch 1/1, Loss: 0.5788148712481971
Test Accuracy: 71.81%
Student accuracy with CE + KD: 71.81%


In [ ]:
torch.save(new_nn_light, '/kaggle/working/nn_kd_1.pth')